In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import scipy.stats as stats


In [2]:
df = pd.read_pickle('Change in collection boxes by support level.p')

# Remove DC
mask = df['GEOID'] != 11001.0
df=df[mask]

step = .05
x = np.arange(0, 1.0+step, step)
to_plot = df.groupby(
                pd.cut(df["r/(rd)"], x)
        )['net'].agg(['sum','count']).reset_index().rename(columns={'sum':'Net Change','count':'Number of counties'})

to_plot['Percent'] = to_plot['r/(rd)'].apply(lambda x: x.left)
to_plot['Color'] = np.where(to_plot['Percent']<.5,'blue','red')

b_mask = to_plot['Percent']<.5

def make_text(row):
    output = ""
    output += 'For this group:<br> * {:,} collection boxes were removed<br>'.format(-row['Net Change'])
    output += ' * from {:,} counties<br>'.format(row['Number of counties'])
    output += ' * where {:.1%} to {:.1%} of voters supported Trump'.format(row['r/(rd)'].left,row['r/(rd)'].right)
    return output
text = to_plot.apply(make_text,axis=1)

data = [go.Bar(
    x=[float(x) + (step/2) for x in to_plot['Percent']],
    y=to_plot['Net Change'],
    marker_color=to_plot['Color'],
    width=step,
    hovertext=text,
    hoverinfo='text'
)]


# plot normal
mu = .5
x = np.arange(0, 1.0+step,step/2)
var = np.var(x)
sigma = np.sqrt(var)
y = stats.norm.pdf(x,mu,sigma) * np.mean(to_plot['Net Change'])
data += [go.Scatter(
    x = x,
    y = y,
    marker_color = 'black',
    name='Normal Distribution',
    hoverinfo='skip'
)]

layout = go.Layout(
    xaxis = {
        'tickformat': ',.0%',
        'range': [0,1],
        'title':'Percent of Voters Supporting Trump'
    },
    yaxis = {
        'title':'Net change in the number of collection boxes'
    },
    title = 'Net change in collection boxes<br>(Excluding Washington D.C.)',
    showlegend=False
)
fig = go.Figure(data=data,layout=layout)

py.plot(fig,filename='net_change.html')

'net_change.html'

In [3]:
to_plot

,r/(rd),Net Change,Number of counties,Percent,Color
0,"(0.0, 0.05]",0,0,0.00,blue
1,"(0.05, 0.1]",-29,3,0.05,blue
2,"(0.1, 0.15]",-40,4,0.10,blue
3,"(0.15, 0.2]",-99,14,0.15,blue
4,"(0.2, 0.25]",-285,12,0.20,blue
5,"(0.25, 0.3]",-63,16,0.25,blue
6,"(0.3, 0.35]",-110,28,0.30,blue
7,"(0.35, 0.4]",-239,39,0.35,blue
8,"(0.4, 0.45]",-222,49,0.40,blue
9,"(0.45, 0.5]",-175,60,0.45,blue


In [4]:
df = pd.read_pickle('Mailbox removal by political affiliation.p')
df

x = np.arange(0, 1.0+step, step)
to_plot = df.groupby(
        pd.cut(df["r/(rd)"], x)
    ).agg({'net':['count','sum'],'total_boxes':'sum'}).reset_index().fillna({'Percent Change':0})

to_plot.columns = ['r/(rd)','n_counties','net','total_boxes']
to_plot['Percent change'] = to_plot['net']/to_plot['total_boxes']
to_plot['bin'] = to_plot['r/(rd)'].apply(lambda x:x.left)
to_plot['color'] = np.where(
    to_plot['bin']<.5,
    'blue','red'
)


def make_text(row):
    output = ""
    output += 'For this group:<br> * {:,} ({:.2%}) collection boxes were removed<br>'\
    .format(-row['net'],-row['Percent change'])
    output += ' * from {:,} counties<br>'.format(row['n_counties'])
    output += ' * where {:.1%} to {:.1%} of voters supported Trump<br>'.format(row['r/(rd)'].left,row['r/(rd)'].right)
    output += ' * {:,} collection boxes were accessible in 2019'.format(row['total_boxes'])
    return output
text = to_plot.apply(make_text,axis=1)

data = go.Bar(
    x = [float(x) + (step/2) for x in to_plot['bin']],
    y = to_plot['Percent change'],
    marker_color=to_plot['color'],
    width=step,
    hovertext=text,
    hoverinfo='text' 
)


layout = go.Layout(
    xaxis = {
        'tickformat': ',.0%',
        'range': [0,1],
        'title':'Percent of Voters Supporting Trump'
    },
    yaxis = {
        'tickformat': ',.1%',
        'title':'Percent change in the number of collection boxes'
    },
    title = 'Percent change in collection boxes<br>(Excluding Washinton D.C)',
    showlegend=False
)

fig = go.Figure(data=data,layout=layout)

py.plot(fig,filename='percent_change.html')

'percent_change.html'

In [5]:
to_plot

,r/(rd),n_counties,net,total_boxes,Percent change,bin,color
0,"(0.0, 0.05]",0,0,0,NaN,0.00,blue
1,"(0.05, 0.1]",3,-29,3067,-0.009455,0.05,blue
2,"(0.1, 0.15]",4,-40,2030,-0.019704,0.10,blue
3,"(0.15, 0.2]",14,-99,8015,-0.012352,0.15,blue
4,"(0.2, 0.25]",12,-285,15064,-0.018919,0.20,blue
5,"(0.25, 0.3]",16,-63,5277,-0.011939,0.25,blue
6,"(0.3, 0.35]",28,-110,9793,-0.011233,0.30,blue
7,"(0.35, 0.4]",39,-239,14522,-0.016458,0.35,blue
8,"(0.4, 0.45]",49,-222,16643,-0.013339,0.40,blue
9,"(0.45, 0.5]",60,-175,14494,-0.012074,0.45,blue
